# This skripts uses the Emotion Classfier to enrich the truths dataset

In [1]:
import pandas as pd
import torch
TEST_FLAG = False
if TEST_FLAG:
    NUM_TRUTHS = 2000
OUTPUT_PATH = "../../Data/ProcessedData/enriched_comments_emotions.tsv"
DEVICE = torch.device("cuda") # Force CPU for testing, change to "cuda" if GPU is available
TEMP_DIR = "./temp_progress"
TEMP_PATH = TEMP_DIR + "/progress.csv"
truths_path = "../../Data/ProcessedData/truths_cleaned.tsv"
df_truths = pd.read_csv(truths_path, sep="\t")
print(df_truths.head())

KeyboardInterrupt: 

In [ ]:
truths_path = "../../Data/ProcessedData/truths_cleaned.tsv"
df_truths = pd.read_csv(truths_path, sep="\t")
print(df_truths.head())

In [ ]:
comments= df_truths["text"].tolist()

if TEST_FLAG:
    comments = comments[:NUM_TRUTHS]
print(comments[0])

In [ ]:
from CommentClassifier.EmotionCommentClassifierLLM import EmotionCommentClassifierLLM

classifier = EmotionCommentClassifierLLM(comments,device=DEVICE)
for comment in comments[:10]:
    print(f"comment: {comment},\n classification: {classifier.classify_one(comment)}\n\n" )


In [ ]:
labels, label_logits = classifier.classify(output_file=TEMP_PATH, batch_size=256, steps_to_save=1000) 
print(labels[:5])
print(label_logits[:5])

In [ ]:
import matplotlib.pyplot as plt

df_classified = pd.read_csv(TEMP_PATH)
df_classified.head()

df_classified["label"].value_counts().sort_index().plot(kind="bar", figsize=(12,5))
plt.title("Label Histogram")
plt.xlabel("Label")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
import random

#example for each label
labels = classifier.labels
for label in labels:
    print(f"{label.upper()}\n")
    examples = df_classified.loc[df_classified["label"] == label, "text"].tolist()
    random.shuffle(examples)
    for i, e in enumerate(examples[:10]):
        print(f"example {i} : {e}\n")

In [ ]:
if not TEST_FLAG:
    df_truths["emotion_label"] = df_classified["label"]
    df_truths["emotion_label_logits"] = df_classified["label_logits"]
    df_truths.to_csv(OUTPUT_PATH, sep="\t", index=False)